# Objects Expression recognition

This notebooks shows some preliminary work on detecting objects expressions with cvlib. It is mainly meant to explore its capabilities and to decide on future research directions. We package our code into a `ammico` package that is imported here:

In [ ]:
import ammico
from ammico import utils as mutils
from ammico import display as mdisplay
import ammico.objects as ob

Set an image path as input file path.

In [ ]:
images = mutils.find_files(
    path="data/",
    limit=10,
)

In [ ]:
mydict = mutils.initialize_dict(images)

## Manually inspect what was detected

To check the analysis, you can inspect the analyzed elements here. Loading the results takes a moment, so please be patient. If you are sure of what you are doing.

In [ ]:
mdisplay.explore_analysis(mydict, identify="objects")

## Detect objects and directly write to csv

In [ ]:
for key in mydict:
    mydict[key] = ob.ObjectDetector(mydict[key]).analyse_image()

Convert the dictionary of dictionarys into a dictionary with lists:

In [ ]:
outdict = mutils.append_data_to_dict(mydict)
df = mutils.dump_df(outdict)

Check the dataframe:

In [ ]:
df.head(10)

Write the csv file:

In [ ]:
df.to_csv("./data_out.csv")